In [ ]:
pip install -q yahoo_fin

In [ ]:
!mkdir -p data results logs


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp  /content/drive/MyDrive/colab/results/* ./results/

Mounted at /content/drive


In [ ]:
pip install  git+https://github.com/ahsank/runml@testv2.01#egg=runml

In [ ]:
import runml.pipeline
from runml import pipeline,findata


In [ ]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAllocHL(df, stop_loss, profit_factor=1):
    #df['Gain_f'] = (2*df['Gain']+df['Gain_l']+df['Gain_h'])/4
    df['Gain_f'] = df[['Gain','Gain_l']].max(axis=1)
    df['Gain_f'] = df[['Gain_h', 'Gain_f']].min(axis=1)
    df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain_f'])
    df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
    return df

In [ ]:
findata.EPOCHS=200
new = []
model = "gstock-12b-test"
tickers = [ 'AEXAF', 'ALSMY', 'AMSSY', 'ASML', 'ADDDF', 'ADYEY',
            'BAK', 'BASFY', 'BESIY',
            'CAAP', 'CDUAF', 'COTY', 'COVTY', 'CDMGF', 'CPG', 'CX',
            'DBOEY',
            'EADSF', 'EIFZF', 'EDNMF', 'ENGGY', 'EQNR', 'ERIC', 'ERJ','ESEA',
            'FANUY', 'FQVLF', 'GLCNF',  'GLOB', 'HCMLY', 'HDELY',
            'HENKY', 'HYMTF', 'HMC', 'KNNGF', 'MELI', 'MGA',
            'RACE', 'LYB', 'LVMUY',
            'MYTE', 'NHYDY', 'NOK', 'NU', 'NVO', 'NXPI',
            'ONON', 'OTGLY', 'PAX', 'POAHF', 'PROSF',
            'RNMBF', 'RYCEF', 'SAFRF', 'SFTBY',
            'ORAN', 'SAP', 'SE', 'SIEGY', 'STLA',
            'TEF', 'TELNF', 'TGLS','THQQF', 'TKAMY', 'TRYIY', 'TSM',
            'VALE', 'VNT', 'YARIY', 'ZURVY']


In [ ]:
findata.EPOCHS=200
model = "val-12b-test"
new = ['DXC', 'FSLR', 'STLD', 'FICO', 'SRE', 'BKR', 'CTAS', 'PEP', 'ADI', 'ROP' ]
tickers = ['AAL', 'ADI', 'ALB', 'ANF', 'APO', 'AQN', 'ARCH', 'ARE',
            'BAH', 'BAX', 'BKR', 'BGFV', 'BX', 'BXC',
            'CBT', 'CC', 'CCI', 'CCJ', 'CF', 'CHK', 'CI', 'CIVI', 'CTAS',
            'CG',  'CLS', 'CMI', 'COKE', 'COP', 'CSL',
            'DAL', 'DD', 'DINO', 'DVN', 'DXC', 'EMN',
            'F', 'FCX', 'FICO', 'FIS', 'FLO', 'FMS', 'FSLR', 'GLW', 'GWW',
            'HIW', 'HSII', 'HPE', 'IP', 'IPI', 'IVZ', 'JWN',
            'KD', 'KHC', 'KMB', 'KVUE',
            'MMM', 'MOD', 'MPC', 'NFG', 'NTR', 'OMC', 'OLN', 'OSK',
            'PARA', 'PCAR', 'PEP', 'PETS', 'PNR', 'POR', 'POWL', 'PSTL', 'PXD',
            'RCL', 'RH', 'ROP',
            'SAFE', 'SCCO', 'SPTN', 'SPG', 'SRE', 'STLD', 'STRL', 'SWK',
            'T', 'TCN', 'TEX', 'TGT', 'TDG', 'TMUS','TWI', 'UAL', 'URI',
            'VRTX', 'VZ', 'WDC', 'WHR', 'WSM', 'XOM', 'YORW']

In [ ]:
findata.EPOCHS=200
model = "cry-1a-test"
tickers = [ 'ADA-USD', 'AVAX-USD', 'BTC-USD', 'DOGE-USD', 'DOT-USD',
            'ETH-USD', 'LINK-USD', 'LTC-USD', 'MATIC-USD', 'SOL-USD', 'XRP-USD']

In [ ]:
findata.EPOCHS=200
model = "etf-1a-test"
tickers = ['ARKF', 'ARKK', 'ARKW', 'DAPP', 'DIA', 'DTEC', 'EEM', 'FPX',
            'ICLN', 'IJR', 'IPO', 'IXC', 'IXN', 'IXP', 'IWM', 'IWO', 'IYZ',
            'JETS', 'MGK', 'MGV', 'MTUM',
            'ONLN', 'QQQ', 'SMH', 'SMOG', 'SPY', 'TDIV',
            'VNQ', 'VT', 'VTI', 'VUG', 'WDIV', 'XITK',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME', 'XNTK', 'XSW' ]

In [ ]:
findata.EPOCHS=200
model = "mcap-11b-test"
tickers = ['AAPL', 'ACN', 'ADBE', 'AMD', 'AMZN', 'AVGO', 'BA', 'BKNG', 'BRK-B',
            'CAT', 'CDNS', 'CRM', 'COST', 'CSCO', 'DE', 'DHR', 'DIS',
            'GOOGL', 'FDX', 'HD', 'INTC', 'IBM', 'ISRG',
            'META', 'MMC', 'MSFT', 'NKE', 'NVDA', 'NOW',
            'ORCL', 'PYPL', 'QCOM', 'SBUX', 'TMO', 'TSLA', 'TXN', 'UNH', 'UPS']

In [ ]:
pipeline.IS_VERBOSE = False
lossfn = "huber_loss"
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(),
                         pipeline.AddVWap(),
                         pipeline.AddMA(200),
                         pipeline.Adj()
                         ]))


df  = pipeline.runModelCombinedVola(tickers, model, mod, False, loss=lossfn)


In [ ]:
addAllocHL(df, 0.10, 1)
df.iloc[(df.Alloc * abs(df.Gain_f)).sort_values(ascending=False).index].head(50)
#df.sort_values('Alloc', ascending=False).head(50)


,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Gain_f,Alloc
12,CVNA,8.24,0.91,1604.58,1509.24,42.78,66.220001,0.55,0.74,1545.26,1486.19,78.879997,0.84,0.74,1127.47,1194.32,38.900002,-0.09,0.55,8.936364
45,NVCR,13.76,0.92,1062.99,1167.08,13.39,21.410000,0.60,0.72,988.57,886.72,19.879999,0.48,0.73,935.53,922.40,15.320000,0.14,0.48,9.033333
2,AI,11.63,0.85,196.27,379.01,26.21,38.910000,0.48,0.61,64.34,255.06,43.090000,0.64,0.87,306.57,494.25,25.350000,-0.03,0.48,8.187500
62,SQ,41.15,0.91,1400.97,1349.06,66.48,95.160004,0.43,0.80,1519.33,1233.11,92.040001,0.38,0.73,1087.22,1323.83,67.900002,0.02,0.38,8.863158
74,WOLF,30.13,0.92,606.40,883.95,33.86,46.439999,0.37,0.71,663.75,726.41,51.310001,0.52,0.74,860.30,677.52,35.150002,0.04,0.37,8.983784
53,PLUG,3.53,0.89,418.38,338.27,3.44,4.750000,0.38,0.74,359.06,297.71,7.950000,1.31,0.69,383.36,267.86,3.720000,0.08,0.38,8.610526
37,LYFT,8.76,0.94,302.80,405.86,12.88,17.250000,0.34,0.72,196.63,317.01,19.160000,0.49,0.77,219.41,345.32,13.410000,0.04,0.34,9.223529
70,U,23.48,0.95,674.84,1000.46,34.59,45.490002,0.32,0.77,760.02,935.80,50.950001,0.47,0.74,523.26,899.17,33.230000,-0.04,0.32,9.343750
77,ZM,66.33,0.91,2173.38,2522.48,70.46,111.570000,0.58,0.71,1574.83,1931.72,90.750000,0.29,0.73,1448.88,1306.96,69.040001,-0.02,0.29,8.789655
73,UPWK,5.99,0.92,264.93,296.47,14.87,19.350000,0.30,0.80,352.70,256.55,19.010000,0.28,0.72,265.03,276.00,13.930000,-0.06,0.28,8.914286


In [ ]:
findata.EPOCHS=200
model = "vols-12b-test"
pending = ['ASTR', 'SMWB', 'APPS']
next = ['TRUP']
tickers = [ 'AIRI', 'AMSC', 'APRN', 'ACHR', 'AUGX', 'AUR',   'ATOM',
            'BKKT', 'BGSF', 'BLNK', 'BSGM',
            'CARS', 'CLOV', 'CRCT', 'CXM', 'CZOO',
            'DLO',  'DM', 'DMTK', 'DPSI', 'EVGO', 'EXPR', 'FSLY', 'FSR', 'FTCH',
            'GOVX', 'GRAB', 'GTE', 'HIMS', 'HIVE',
            'INLX',  'INVZ', 'JOBY',  'KIND', 'KULR',
            'LAZR', 'LUMN', 'NNOX', 'MAX', 'MTTR',
            'NEPH', 'NSTG', 'OMQS', 'ONDS', 'OPEN', 'OTLY', 'OUST',
            'PAYO', 'PTON', 'REAL', 'REI', 'RIG', 'RKLB', 'RVYL',
            'SAVE', 'SFIX', 'SOUN', 'STEM', 'SFT',  'TRUP', 'TELL',
            'VTEX', 'YEXT', 'WKME']
future = []

In [ ]:
pending = ['PAR']
findata.EPOCHS=200
model = "ipos-12a-test"
next = ['AYX', 'FIVE']
tickers = ['ABNB', 'ACLS' ,'AI', 'AMBA', 'APP', 'AYX',
           'BILL', 'BMBL', 'CELH', 'CFLT', 'CHGG', 'CRWD', 'CVNA',
           'DASH', 'DBX', 'DDOG', 'DOCN', 'DOCS', 'DOCU', 'DT', 'DXCM',
           'ENPH', 'ESTC', 'ETSY', 'EXPE', 'FIVE', 'FOUR',
           'GFS', 'GTLB', 'GLBE', 'GOGO',
           'INDI', 'INMD', 'INTA', 'IOT', 'IRDM', 'JKS', 'LYFT',
           'MBLY', 'MDB',  'MNDY', 'MNST', 'MPWR',  'MXL',
           'MTCH',  'NVCR', 'NTNX', 'OKTA', 'OLED',
           'PERI', 'PANW',  'PAYX', 'PD', 'PLUG', 'PI', 'PINS', 'PUBM',
           'RBLX', 'SMCI', 'SNAP', 'SNOW', 'SPLK', 'SQ', 'SPOT',
           'TEAM', 'TDOC', 'TOST', 'TRIP', 'TTD', 'TWLO',
           'U','UI', 'UBER', 'UPWK', 'WOLF', 'VEEV', 'Z', 'ZM', 'ZS']

In [ ]:
findata.EPOCHS=200
model = "cetf-11b-test"
tickers = ['ARGT', 'EWD', 'FLN', 'FM',
            'ECH', 'EWA', 'EWI', 'EWC', 'EWM', 'EWP', 'EWW', 'EWY', 'EWZ',
            'IDX', 'ILF', 'KEMQ', 'THD', 'TUR', 'XCEM']

In [ ]:
findata.EPOCHS=200
model = "com-12a-test"
tickers = [ 'GLD', 'ISHG', 'PDBC', 'SLV', 'SOYB',
            'TIP', 'TLT', 'UUP', 'VTIP', 'WEAT']

In [ ]:
lossfn = "huber_loss"
pipeline.IS_VERBOSE = False
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(),
                         pipeline.AddVWap(),
                         pipeline.AddMA(200),
                         pipeline.Adj()
                         ]))


#df  = pipeline.runModelCombined(tickers, 'ipos2-3a', mod, False, loss=lossfn, output='low', trading=pipeline.LowTrading)[0]
#df  = pipeline.runModelCombined(tickers, 'ipos2-3a', mod, False, loss=lossfn, output='high', trading=pipeline.HighTrading)[0]
df  = pipeline.runModelCombinedVola(tickers, model, mod, True, loss=lossfn)

In [ ]:
addAllocHL(df, 0.10, 1)
df.iloc[(df.Alloc * abs(df.Gain_f)).sort_values(ascending=False).index].head(50)
#df.sort_values('Alloc', ascending=False).head(50)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Gain_f,Alloc
3,DAPP,2.94,0.88,94.98,101.09,8.34,10.520000,0.26,0.68,69.48,90.28,11.900000,0.43,0.78,79.86,127.57,8.310000,-0.00,0.26,8.338462
2,ARKW,37.88,0.93,837.37,609.35,70.17,83.870003,0.20,0.89,871.14,619.95,87.669998,0.25,0.73,608.78,697.48,65.489998,-0.07,0.20,8.950000
21,ONLN,26.87,0.93,318.37,314.04,34.13,40.029999,0.17,0.77,357.19,331.30,44.310001,0.30,0.73,204.67,258.99,33.330002,-0.02,0.17,8.888235
1,ARKK,30.85,0.90,627.68,599.94,47.38,55.380001,0.17,0.76,483.60,452.33,59.900002,0.26,0.76,561.62,739.58,44.750000,-0.06,0.17,8.411765
0,ARKF,14.11,0.96,215.96,273.70,25.57,29.480000,0.15,0.74,200.09,210.61,30.889999,0.21,0.72,196.49,153.25,23.670000,-0.07,0.15,9.333333
8,ICLN,8.26,0.96,110.85,133.87,14.54,16.629999,0.14,0.72,86.68,76.10,17.950001,0.23,0.69,76.43,49.03,13.970000,-0.04,0.14,9.314286
10,IPO,22.34,0.94,295.73,309.34,35.24,40.090000,0.14,0.82,271.11,253.58,41.590000,0.18,0.71,261.00,319.18,33.080002,-0.06,0.14,8.971429
32,XITK,85.93,0.95,1055.88,769.39,142.55,156.899994,0.10,0.79,1001.83,864.88,161.699997,0.13,0.69,797.24,857.56,138.559998,-0.03,0.10,9.000000
35,XLE,20.76,0.92,353.66,181.70,82.68,75.440002,-0.09,0.81,441.14,314.84,80.559998,-0.03,0.68,238.79,185.43,74.769997,-0.10,-0.09,8.311111
24,SMOG,50.69,0.95,704.17,645.35,102.77,110.919998,0.08,0.78,581.95,609.89,112.510002,0.09,0.66,550.64,397.02,99.959999,-0.03,0.08,8.875000


In [ ]:
pip install seaborn

In [ ]:
import seaborn as sns

Todos:



1.   Load previously saved model before training
2.   Allow eval only without test/train spliting
3.   Incremental data load
4.    Add S&P, QQQ etc to the model
5. Save model to google drive






In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
  !cp  /content/drive/MyDrive/colab/results/* ./results/
else:
  ! cp ~/Google\ Drive/My\ Drive/colab/results/* ./results/

Run following in local Jupyter

Run following 2 cells for colab to import libraries

Run following if the library code is modified locally

In [ ]:
!git clone https://github.com/ahsank/runml

In [ ]:
import os
import sys
path = os.path.expanduser('~/src/runml')
print(path)
if not path in sys.path:
  sys.path.append(path)

from runml import pipeline,findata

/Users/ahsank/src/runml


In [ ]:
import importlib
import sys
path = 'runml'
if not path in sys.path:
  sys.path.append(path)

from runml import pipeline,findata


In [ ]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAlloc(df, stop_loss, profit_factor=2):
  df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain'])
  df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
  return df

In [ ]:
from importlib import reload
reload(findata)
reload(pipeline)

<module 'runml.pipeline' from '/Users/ahsank/src/runml/runml/pipeline.py'>

In [ ]:
import pandas as pd
df1 = pipeline.fetch_data('ABNB')
#df1.sort_index(inplace=True)
df1 = df1.round(3)

df2 = pd.read_csv('data/ABNB_2023-04-15.csv', index_col=0)
df2.index =  pd.to_datetime(df2.index)
# df2 = df2.round(3)
#data2 = data.tail(1000)
#pd.concat([df1,df2]).drop_duplicates(keep=False)
#df1.compare(df2)
#df2.index.dtype
df1

In [ ]:
display(df.sort_values('Gain', ascending=False))
display(df[["Buy", "Sell", "Total"]].sum()/len(df))


In [ ]:
results['ABNB'].pdata.__dict__.keys()

In [ ]:
findata.EPOCHS=10
tickers1 = ['ARKK', 'ARKW', 'DAPP', 'DTEC', 'EEM', 'FPX',
            'GBTC', 'GLD',
            'ICLN', 'IJR', 'IPO', 'IPOS', 'IWM',
            'JETS', 'KEMQ',
            'MGK', 'MGV', 'MOAT', 'MTUM',
            'QQQ', 'SLV','SMH', 'SMOG',
            'SPY', 'VNQ', 'VT', 'VTI', 'WDIV',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME' ]
mod = pipeline.RateReturnOnly(pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap()]))
df1 = pipeline.runModelCombined(tickers1, 'etf2b', mod, True)

In [ ]:
df1.sort_values('Gain', ascending=False)

In [ ]:
findata.EPOCHS=200
tickers3 = [
           'AHT', 'AMSC', 'ASTR', 'ATOM',
           'BKKT', 'BGFV', 'BGSF', 'CBNT', 'CLOV',
           'DNMR', 'ERJ', 'EVGO',
           'FSLY',  'FTCH', 'GOGO', 'HIVE',
           'ILAL', 'INLX', 'JMIA', 'JOBY',  'KULR', 'MTTR',
           'MYTE', 'NEPH', 'ONDS', 'MQ',
           'PETS', 'PTON', 'SFIX', 'SFT', 'STNE',
           'ULH', 'VRAR', 'WISH']
lossfn = "huber_loss"
# lossfn = "mean_squared_error"
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df3, results = pipeline.runModelCombined(tickers3, 'vols', mod, False, loss=lossfn)

In [ ]:
df3

In [ ]:
import pandas as pd
seq = list()
for ticker, result in results.items():
    tempdf = result.final_df
    tempdf['predicted_rate'] = (tempdf['adjclose_15']-tempdf['adjclose'])/tempdf['adjclose']
    tempdf['true_rate'] = (tempdf['true_adjclose_15']-tempdf['adjclose'])/tempdf['adjclose']
    seq.append(tempdf[['ticker', 'date', "predicted_rate", 'true_rate']])

detailstat = pd.concat(seq)



In [ ]:
normalstat = detailstat[detailstat['predicted_rate'].between(-2,2) & detailstat['true_rate'].between(-2,2)]
largestat = normalstat[~ normalstat['predicted_rate'].between(-0.25,0.25) & ~ normalstat['true_rate'].between(-0.25,0.25)]
largestat.sample(10)

In [ ]:
ax = largestat.plot.scatter(x='predicted_rate', y='true_rate',c='DarkBlue', figsize=(10,10))
# ax.set_yscale('log')
# ax.set_xscale('log')

In [ ]:
# import matplotlib.pyplot as plt

# plt.hist([normalstat['predicted_rate'],normalstat['predicted_rate']],
#          bins=100, range=(-1,1), color = ['r','g'])

corr = largestat[["predicted_rate", "true_rate"]].corr()

corr.style.background_gradient(cmap='coolwarm')

In [ ]:

sns.regplot(x=largestat["predicted_rate"], y=largestat["true_rate"])


In [ ]:
sns.heatmap(normalstat[["predicted_rate", "true_rate"]].corr(), annot = True, fmt='.2g',cmap= 'coolwarm')

In [ ]:
!rsync -av --ignore-existing results/* /content/drive/MyDrive/colab/results/


sending incremental file list
etf-1a-test-adjclose-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
etf-1a-test-high-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
etf-1a-test-low-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5

sent 9,648,696 bytes  received 73 bytes  19,297,538.00 bytes/sec
total size is 202,383,216  speedup is 20.98
